In [40]:
#!pip install tensorflow-gpu
#!pip install tensorflow
#!pip install keras

In [41]:
from PIL import Image
import random
import warnings
import os
import shutil
from PIL import ImageFile

from tensorflow import keras

import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score
from sklearn.model_selection import StratifiedKFold
keras.__version__

'2.7.0'

In [42]:
base_dir = r"E:\Koding\Python\Kuliah\S2\AI\Projek\Dataset"

training_dir = os.path.join(base_dir, 'Training')
test_dir = os.path.join(base_dir, 'Testing')
validation_dir = os.path.join(base_dir, 'Validation')

In [43]:
sourceFiles = []
classLabels = ['Pneumonia', 'Covid', 'Normal']
picture_size = (150, 150)
batch_size = 32
epoch = 10

In [44]:
def transferBetweenFolders(source, dest, splitRate):
    global sourceFiles
    sourceFiles = os.listdir(source)
    if (len(sourceFiles) != 0):
        transferFileNumbers = int(len(sourceFiles) * splitRate)
        transferIndex = random.sample(range(0, len(sourceFiles)), transferFileNumbers)
        for eachIndex in transferIndex:
            shutil.move(source + str(sourceFiles[eachIndex]), dest + str(sourceFiles[eachIndex]))
    else:
        print("No file moved. Source empty!")


def transferAllClassBetweenFolders(source, dest, splitRate):
    for label in classLabels:
        transferBetweenFolders(base_dir + '/' + source + '/' + label + '/',
                                base_dir + '/' + dest + '/' + label + '/',
                               splitRate)

In [45]:
# First, check if test folder is empty or not, if not transfer all existing files to train
transferAllClassBetweenFolders('Testing', 'Training', 1.0)
transferAllClassBetweenFolders('Validation', 'Training', 1.0)
# Now, split some part of train data into the test folders.
transferAllClassBetweenFolders('Training', 'Testing', 0.20)

In [46]:
X = []
Y = []

def prepareNameWithLabels(folderName):
    sourceFiles = os.listdir(base_dir + '/Training/' + folderName)
    for val in sourceFiles:
         X.append(val)
         if (folderName == classLabels[0]):
             Y.append(0)
         elif (folderName == classLabels[1]):
             Y.append(1)
         else:
             Y.append(2)
             
# Organize file names and class labels in X and Y variables
for i in range(len(classLabels)):
    prepareNameWithLabels(classLabels[i])

X = np.asarray(X)
Y = np.asarray(Y)

In [47]:
from keras.models import Model
from keras.layers import Conv2D, MaxPool2D,  \
    Dropout, Dense, Input, concatenate,      \
    GlobalAveragePooling2D, AveragePooling2D,\
    Flatten

def inception_module(x, 
                     activation,
                     filters_1x1,
                     filters_3x3_reduce,
                     filters_3x3,
                     filters_5x5_reduce,
                     filters_5x5,
                     filters_pool_proj,
                     name=None):
    
    conv_1x1 = Conv2D(filters_1x1, (1, 1), padding='same', activation=activation, 
                      kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    
    conv_3x3 = Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation=activation, 
                      kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_3x3 = Conv2D(filters_3x3, (3, 3), padding='same', activation=activation, 
                      kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_3x3)

    conv_5x5 = Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation=activation, 
                      kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_5x5 = Conv2D(filters_5x5, (5, 5), padding='same', activation=activation, 
                      kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_5x5)

    pool_proj = MaxPool2D((3, 3), strides=(1, 1), padding='same')(x)
    pool_proj = Conv2D(filters_pool_proj, (1, 1), padding='same', activation=activation, 
                       kernel_initializer=kernel_init, bias_initializer=bias_init)(pool_proj)

    output = concatenate([conv_1x1, conv_3x3, conv_5x5, pool_proj], axis=3, name=name)
    
    return output

kernel_init = keras.initializers.glorot_uniform()
bias_init = keras.initializers.Constant(value=0.2)

input_layer = Input(shape=(*picture_size, 3))
activation = 'relu'

x = Conv2D(64, (7, 7), padding='same', strides=(2, 2), activation=activation, 
           name='conv_1_7x7/2', kernel_initializer=kernel_init, bias_initializer=bias_init)(input_layer)
x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_1_3x3/2')(x)
x = Conv2D(64, (1, 1), padding='same', strides=(1, 1), activation=activation, name='conv_2a_3x3/1')(x)
x = Conv2D(128, (3, 3), padding='same', strides=(1, 1), activation=activation, name='conv_2b_3x3/1')(x)
x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_2_3x3/2')(x)

x = inception_module(x,
                     activation,
                     filters_1x1=64,
                     filters_3x3_reduce=96,
                     filters_3x3=128,
                     filters_5x5_reduce=16,
                     filters_5x5=32,
                     filters_pool_proj=32,
                     name='inception_3a')

x = inception_module(x,
                     activation,
                     filters_1x1=128,
                     filters_3x3_reduce=128,
                     filters_3x3=192,
                     filters_5x5_reduce=32,
                     filters_5x5=96,
                     filters_pool_proj=64,
                     name='inception_3b')

x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_3_3x3/2')(x)

x = inception_module(x,
                     activation,
                     filters_1x1=192,
                     filters_3x3_reduce=96,
                     filters_3x3=208,
                     filters_5x5_reduce=16,
                     filters_5x5=48,
                     filters_pool_proj=64,
                     name='inception_4a')


x1 = AveragePooling2D((5, 5), strides=3)(x)
x1 = Conv2D(128, (1, 1), padding='same', activation=activation)(x1)
x1 = Flatten()(x1)
x1 = Dense(64, activation=activation)(x1)
x1 = Dropout(0.3)(x1)
x1 = Dense(len(classLabels), activation='softmax', name='auxilliary_output_1')(x1)

x = inception_module(x,
                     activation,
                     filters_1x1=160,
                     filters_3x3_reduce=112,
                     filters_3x3=224,
                     filters_5x5_reduce=24,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4b')

x = inception_module(x,
                     activation,
                     filters_1x1=128,
                     filters_3x3_reduce=128,
                     filters_3x3=256,
                     filters_5x5_reduce=24,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4c')

x = inception_module(x,
                     activation,
                     filters_1x1=112,
                     filters_3x3_reduce=144,
                     filters_3x3=288,
                     filters_5x5_reduce=32,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4d')


x2 = AveragePooling2D((5, 5), strides=3)(x)
x2 = Conv2D(128, (1, 1), padding='same', activation=activation)(x2)
x2 = Flatten()(x2)
x2 = Dense(64, activation=activation)(x2)
x2 = Dropout(0.3)(x2)
x2 = Dense(len(classLabels), activation='softmax', name='auxilliary_output_2')(x2)

x = inception_module(x,
                     activation,
                     filters_1x1=256,
                     filters_3x3_reduce=160,
                     filters_3x3=320,
                     filters_5x5_reduce=32,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_4e')

x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_4_3x3/2')(x)

x = inception_module(x,
                     activation,
                     filters_1x1=256,
                     filters_3x3_reduce=160,
                     filters_3x3=320,
                     filters_5x5_reduce=32,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_5a')

x = inception_module(x,
                     activation,
                     filters_1x1=384,
                     filters_3x3_reduce=192,
                     filters_3x3=384,
                     filters_5x5_reduce=48,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_5b')

x = GlobalAveragePooling2D(name='avg_pool_5_3x3/1')(x)
x = Dropout(0.2)(x)
x = Dense(len(classLabels), activation='softmax', name='output')(x)

cnn = Model(input_layer, [x, x1, x2], name='inception_v1')

learning_rate = 0.001
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
losses = 'categorical_crossentropy'
cnn.compile(optimizer=optimizer, loss=losses, metrics=['accuracy'])
cnn.summary()

Model: "inception_v1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv_1_7x7/2 (Conv2D)          (None, 75, 75, 64)   9472        ['input_4[0][0]']                
                                                                                                  
 max_pool_1_3x3/2 (MaxPooling2D  (None, 38, 38, 64)  0           ['conv_1_7x7/2[0][0]']           
 )                                                                                                
                                                                                       

In [48]:
filepath = 'inception_v1.h5'
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss',
                                                 verbose=1, save_weights_only=False,
                                                 save_best_only=True, mode='min')

logdir = os.path.join(base_dir, 'logs/')
tfboard = keras.callbacks.TensorBoard(log_dir=logdir)

callbacks_list = [checkpoint, tfboard]

In [ ]:
# ===============Stratified K-Fold======================
skf = StratifiedKFold(n_splits=5, shuffle=True)
skf.get_n_splits(X, Y)
foldNum=0
history_loss = []
history_acc = []
history_val_loss = []
history_val_acc = []
for train_index, val_index in skf.split(X, Y):
    #First cut all images from validation to train (if any exists)
    transferAllClassBetweenFolders('Validation', 'Training', 1.0)
    foldNum+=1
    print("Results for fold",foldNum)

    X_train, X_val = X[train_index], X[val_index]
    Y_train, Y_val = Y[train_index], Y[val_index]
    # Move validation images of this fold from train folder to the validation folder
    for eachIndex in range(len(X_val)):
        classLabel=''
        if(Y_val[eachIndex]==0):
            classLabel=classLabels[0]
        elif(Y_val[eachIndex]==1):
            classLabel=classLabels[1]
        else:
            classLabel=classLabels[2]
        #Then, copy the validation images to the validation folder
        shutil.move(base_dir+'/Training/'+classLabel+'/'+X_val[eachIndex],
                    base_dir+'/Validation/'+classLabel+'/'+X_val[eachIndex])

    picture_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1. / 255,
                                           rotation_range=20,
                                           shear_range=0.2,
                                           zoom_range=0.2,
                                           horizontal_flip=True)
    
    training_set = picture_datagen.flow_from_directory(training_dir,
                                                       shuffle=True,
                                                       target_size=picture_size,
                                                       batch_size=batch_size,
                                                       class_mode='categorical')

    # Preprocessing the Test set
    test_set = picture_datagen.flow_from_directory(validation_dir,
                                                   shuffle=True,
                                                   target_size=picture_size,
                                                   batch_size=batch_size,
                                                   class_mode='categorical')


    # Training the CNN on the Training set and evaluating it on the Test set
    fit = cnn.fit(training_set,
                    epochs=epoch,
                    validation_data=test_set)
    
    for i in range(epoch):
      history_loss.append(fit.history['output_loss'][i])
      history_acc.append(fit.history['output_accuracy'][i])
      history_val_loss.append(fit.history['val_output_loss'][i])
      history_val_acc.append(fit.history['val_output_accuracy'][i])

No file moved. Source empty!
No file moved. Source empty!
No file moved. Source empty!
Results for fold 1
Found 1920 images belonging to 3 classes.
Found 480 images belonging to 3 classes.
Epoch 1/10
60/60 [==============================] - 311s 5s/step - loss: 3.6090 - output_loss: 1.3531 - auxilliary_output_1_loss: 1.1281 - auxilliary_output_2_loss: 1.1279 - output_accuracy: 0.3500 - auxilliary_output_1_accuracy: 0.3380 - auxilliary_output_2_accuracy: 0.3339 - val_loss: 3.3353 - val_output_loss: 1.1356 - val_auxilliary_output_1_loss: 1.0981 - val_auxilliary_output_2_loss: 1.1016 - val_output_accuracy: 0.3333 - val_auxilliary_output_1_accuracy: 0.3333 - val_auxilliary_output_2_accuracy: 0.3333
Epoch 2/10
60/60 [==============================] - 315s 5s/step - loss: 3.1851 - output_loss: 1.1166 - auxilliary_output_1_loss: 1.0091 - auxilliary_output_2_loss: 1.0593 - output_accuracy: 0.3625 - auxilliary_output_1_accuracy: 0.4729 - auxilliary_output_2_accuracy: 0.4125 - val_loss: 2.8056 -

In [ ]:
import pylab as pl
pl.figure()

pl.subplot(121)
pl.plot(history_acc)
pl.title('Accuracy:')
pl.plot(history_val_acc)
pl.legend(('Train', 'Val'))

pl.subplot(122)
pl.plot(history_loss)
pl.title('Loss:')
pl.plot(history_val_loss)
pl.legend(('Train', 'Val'))

In [ ]:
pl.figure()

pl.subplot(121)
pl.ylim([0,1])
pl.plot(history_acc)
pl.title('Accuracy train:')

pl.subplot(122)
pl.ylim([0,1])
pl.title('Accuracy Val:')
pl.plot(history_val_acc)

In [ ]:
pl.figure()

pl.subplot(121)
pl.plot(history_loss)
pl.title('Loss Train:')

pl.subplot(122)
pl.title('Loss Val:')
pl.plot(history_val_loss)

In [ ]:
modelpath = os.path.join(base_dir, filepath)
cnn.save(modelpath)

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, confusion_matrix, roc_curve, auc, recall_score
import pylab as pl

def my_metrics(y_true, y_pred, classes, predictions):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average="macro")
    # f1Score=f1_score(y_true, y_pred, average='weighted')
    roc_auc = roc_auc_score(classes, predictions, average='macro', multi_class='ovr')
    print("Accuracy  : {}".format(accuracy))
    print("Precision : {}".format(precision))
    print("Recall : {}".format(recall))
    print("AUC : {}".format(roc_auc))
    cm = confusion_matrix(y_true, y_pred)
    print(cm)
    return accuracy, precision, roc_auc

In [ ]:
# menghitung akurasi dari training data

picture_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1. / 255,
                                           shear_range=0.2,
                                           zoom_range=0.2,
                                           horizontal_flip=True)
    
training_set = picture_datagen.flow_from_directory(test_dir,
                                                       shuffle=True,
                                                       target_size=picture_size,
                                                       batch_size=batch_size,
                                                       class_mode='categorical')
score = cnn.evaluate(training_set)

print('Loss: {:0.2f}%'.format(score[1] * 100))
print('Accuracy: {:0.2f}%'.format(score[4] * 100))

In [ ]:
# menghitung akurasi dari training data

picture_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1. / 255,
                                           shear_range=0.2,
                                           zoom_range=0.2,
                                           horizontal_flip=True)
    
training_set = picture_datagen.flow_from_directory(test_dir,
                                                       shuffle=True,
                                                       target_size=picture_size,
                                                       batch_size=batch_size,
                                                       class_mode='categorical')

predictions = cnn.predict(training_set, verbose=1)

yPredictions = np.argmax(predictions[0], axis=1)
true_classes = training_set.classes
classes = []
for i in true_classes:
    if(i == 0):
        classes.append([1,0,0])
    elif(i == 1):
        classes.append([0,1,0])
    else:
        classes.append([0,0,1])

my_metrics(true_classes, yPredictions, classes, predictions[0])

score = cnn.evaluate(training_set)

print('Loss: {:0.2f}%'.format(score[1] * 100))
print('Accuracy: {:0.2f}%'.format(score[4] * 100))

In [ ]:
print(len(predictions[1]))

predictions[1]